### Setting up

In [1]:
with open('./config.env', 'w') as f:
    f.write("")

Import necessary packages from did module

In [2]:
import did
from did import DID, DIDDocument, Query
from did.database import GridFSBinary, Mongo
from did.versioning import hash_document, hash_snapshot
from did import set_db_configuration, get_db_configuration, settings

Attach the mongo_driver to create an instance of the DID class

In [3]:
mongo_driver = Mongo(connection_string="mongodb://localhost:27017", hard_reset_on_init=True)
gridfs = GridFSBinary(gridfs_connection="mongodb://localhost:27017")
my_did = DID(driver = mongo_driver, binary_driver = gridfs, auto_save = False)

Or alternatively, we can set up a default configuration such that an instance of the DID class can be automatically created at when we import the did package. Nothing has been setup yet, so we expect to get an error when we call get_db_configuration(), which prompts us to to set the env variable 'DB' as a proper key-value pairs

In [4]:
try:
    get_db_configuration()
except Exception as ex:
    print("Error: {}".format(ex))
    print()

Error: Cannot find global variable DB



In [5]:
did.session

DBNotInitialized(Call db.sessin.init to create database connection)

Let's set up a proper database configuration

In [6]:
set_db_configuration('mongodb', {'connection_string': 'mongodb://localhost:27017', 'hard_reset_on_init': True}, 'gridfs', {'gridfs_connection': 'mongodb://localhost:27017'})

{'db': {'type': 'mongodb',
  'args': {'connection_string': 'mongodb://localhost:27017',
   'hard_reset_on_init': True}},
 'binary': {'type': 'gridfs',
  'args': {'gridfs_connection': 'mongodb://localhost:27017'}}}

Verify that the configuration is indeed valid

In [7]:
get_db_configuration()

({'type': 'mongodb',
  'args': {'connection_string': 'mongodb://localhost:27017',
   'hard_reset_on_init': True}},
 {'type': 'gridfs',
  'args': {'gridfs_connection': 'mongodb://localhost:27017'}})

Try to create an instance of DID class again by calling session.init()

In [8]:
did.session.init()

Now we can use did.session to read, write and update DIDDocument from the database. In fact did.session will be accessable next time we import did

In [9]:
did.session

### MongoDB Demo

Create mock documents

In [10]:
data =  [
        {
            'base': {
                'id': '0',
                'session_id': '2387492',
                'name': 'A',
                'datestamp': '2020-10-28T08:12:20+0000',
                'snapshots': [],
                'records': [],
            },
            'depends_on': [],
            'dependencies': [],
            'binary_files': [],
            'document_class': {
                'definition': '$NDIDOCUMENTPATH/ndi_document_app.json',
                'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json',
                'class_name': 'ndi_document_app',
                'property_list_name': 'app',
                'class_version': 1,
                'superclasses': [{
                    'definition': '$NDIDOCUMENTPATH/base_document.json'
                }],
            },
            'app': {
                'a': True,
                'b': True
            },
        },
        {
            'base': {
                'id': '1',
                'session_id': '2387492',
                'name': 'B',
                'datestamp': '2020-10-28T08:12:20+0000',
                'snapshots': [],
                'records': [],
            },
            'depends_on': [],
            'dependencies': [],
            'binary_files': [],
            'document_class': {
                'definition': '$NDIDOCUMENTPATH/ndi_document_app.json',
                'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json',
                'class_name': 'ndi_document_app',
                'property_list_name': 'app',
                'class_version': 1,
                'superclasses': [{
                    'definition': '$NDIDOCUMENTPATH/base_document.json'
                }],
            },
            'app': {
                'a': True,
                'b': False
            },
        },
        {
            'base': {
                'id': '2',
                'session_id': '2387492',
                'name': 'C',
                'datestamp': '2020-10-28T08:12:20+0000',
                'snapshots': [],
                'records': [],
            },
            'depends_on': [],
            'dependencies': [],
            'binary_files': [],
            'document_class': {
                'definition': '$NDIDOCUMENTPATH/ndi_document_app.json',
                'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json',
                'class_name': 'ndi_document_app',
                'property_list_name': 'app',
                'class_version': 1,
                'superclasses': [{
                    'definition': '$NDIDOCUMENTPATH/base_document.json'
                }],
            },
            'app': {
                'a': False,
                'b': False
            },
        },
]

docs = [DIDDocument(data=mock) for mock in data]

Helper function used to print data from a list of DIDDocument

In [11]:
def print_data(docs):
    for doc in docs:
        print(doc.data)
        print()

Add a new document to the database

In [12]:
did.session.add(docs[1])
docs[1].data

{'base': {'id': '1',
  'session_id': '2387492',
  'name': 'B',
  'datestamp': '2020-10-28T08:12:20+0000',
  'snapshots': ['60747e3ad50b3b0b66a6f5a6'],
  'records': ['738bf14851743e2629fe4d7ae647412172db71a7dbeab83c3a2fde887e3c3db2']},
 'depends_on': [],
 'dependencies': [],
 'binary_files': [],
 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json',
  'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json',
  'class_name': 'ndi_document_app',
  'property_list_name': 'app',
  'class_version': 1,
  'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]},
 'app': {'a': True, 'b': False}}

We can find the document we have added by id

In [13]:
print_data(did.session.find_by_id("1"))

{'base': {'id': '1', 'session_id': '2387492', 'name': 'B', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['738bf14851743e2629fe4d7ae647412172db71a7dbeab83c3a2fde887e3c3db2']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'ndi_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': False}}



We can delete the document that we have just added

In [14]:
my_did.delete_by_id("1")

We can look for an document by matching a field in the document against a regular expression. In this case, it should returns am empty list

In [15]:
did.session.find(Query("base.id").match(".*"))

[]

Let's add another document to the database

In [16]:
did.session.add(docs[0])
hash_for_doc0 = hash_document(docs[0])

We can look for a document by its hashes

In [17]:
print(did.session.find_by_hash(hash_for_doc0 ).data)

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['082762677bb6364d8562288f3472a95de9b706a7e4f87f4b43f149ce570da5b8']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'ndi_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}


We can esaily find all the documents in the current working snapshot by not passing any argument to the find method

In [18]:
print_data(did.session.find())

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['082762677bb6364d8562288f3472a95de9b706a7e4f87f4b43f149ce570da5b8']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'ndi_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



Let's try updating the content of a document by changing 'class name' field from 'ndi_document_app' to 'did_document_app'

In [19]:
print(docs[0].data)
docs[0].data['document_class']['class_name'] = 'did_document_app'

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['082762677bb6364d8562288f3472a95de9b706a7e4f87f4b43f149ce570da5b8']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'ndi_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}


We can easily update the document by passing that document as an argument to the update method. It will find the document in the current working snapshot which has the same id

In [20]:
did.session.update(docs[0])

We can verify that the content has been updated

In [21]:
print_data(did.session.find_by_id("0"))

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



We can save the modifications we have made into the database as a new commit

In [22]:
did.session.save("This is my first commit")

Verify that we have everything we have added

In [23]:
print_data(did.session.find())

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



We can access our database modification history

In [24]:
did.session.get_history()

[{'commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'id': ObjectId('60747e3cd50b3b0b66a6f5aa'),
  'message': 'This is my first commit',
  'parent_commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'snapshot_id': ObjectId('60747e3ad50b3b0b66a6f5a6'),
  'timestamp': '2021-04-12T13:07:08.341218',
  'type': 'COMMIT'},
 {'commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'id': ObjectId('60747e39d50b3b0b66a6f5a4'),
  'message': 'Database initialized',
  'parent_commit_hash': None,
  'snapshot_id': ObjectId('60747e39d50b3b0b66a6f5a3'),
  'timestamp': '2021-04-12T13:07:05.970267',
  'type': 'COMMIT'}]

We can delete a document from the snapshot by its document id

In [25]:
did.session.delete_by_id('0')

Verify that the document has indeed being deleted

In [26]:
print_data(did.session.find())

We can revert all changes made to the database. In this case, calling revert undo the deletion of the document

In [27]:
did.session.revert()

Double check that we still have the document

In [28]:
print_data(did.session.find(Query("base.id").match(".*")))

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



The commit history should not be modified if we revert a working snapshot

In [29]:
did.session.get_history()

[{'commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'id': ObjectId('60747e3cd50b3b0b66a6f5aa'),
  'message': 'This is my first commit',
  'parent_commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'snapshot_id': ObjectId('60747e3ad50b3b0b66a6f5a6'),
  'timestamp': '2021-04-12T13:07:08.341218',
  'type': 'COMMIT'},
 {'commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'id': ObjectId('60747e39d50b3b0b66a6f5a4'),
  'message': 'Database initialized',
  'parent_commit_hash': None,
  'snapshot_id': ObjectId('60747e39d50b3b0b66a6f5a3'),
  'timestamp': '2021-04-12T13:07:05.970267',
  'type': 'COMMIT'}]

Delete all the documents in the database and save it. This should push the database back to its initialization state

In [30]:
did.session.delete_by_id("0")
print_data(my_did.find())
did.session.save("Going back to the initialization state")

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



Note that our newest commit points to the same snapshot_id as our first commit

In [31]:
did.session.get_history()

[{'commit_hash': '27fa5c0dd95905d3d4860bc392f7f80b88f4ce2d816e9445599e0841b26e51a6',
  'id': ObjectId('60747e3dd50b3b0b66a6f5ad'),
  'message': 'Going back to the initialization state',
  'parent_commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'snapshot_id': ObjectId('60747e39d50b3b0b66a6f5a3'),
  'timestamp': '2021-04-12T13:07:09.385321',
  'type': 'COMMIT'},
 {'commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'id': ObjectId('60747e3cd50b3b0b66a6f5aa'),
  'message': 'This is my first commit',
  'parent_commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'snapshot_id': ObjectId('60747e3ad50b3b0b66a6f5a6'),
  'timestamp': '2021-04-12T13:07:08.341218',
  'type': 'COMMIT'},
 {'commit_hash': 'a8118917361a8ff6f6bd182e31ec84c9ce8c40b5a489f9d056704dffc85a2cc5',
  'id': ObjectId('60747e39d50b3b0b66a6f5a4'),
  'message': 'Database initialized',
  'parent_commit_hash': None,
  'snapshot_id': Ob

Make another commit

In [32]:
did.session.add(docs[1])
did.session.add(docs[0])
print_data(my_did.find())
did.session.save("My third commit!")

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



In [33]:
did.session.get_history()

[{'commit_hash': '202f01a4a0fcbabd210907ed0fa76ccfb65218086b7dfc2a634688e9e89202c6',
  'id': ObjectId('60747e3dd50b3b0b66a6f5b0'),
  'message': 'My third commit!',
  'parent_commit_hash': '27fa5c0dd95905d3d4860bc392f7f80b88f4ce2d816e9445599e0841b26e51a6',
  'snapshot_id': ObjectId('60747e3dd50b3b0b66a6f5ae'),
  'timestamp': '2021-04-12T13:07:09.829421',
  'type': 'COMMIT'},
 {'commit_hash': '27fa5c0dd95905d3d4860bc392f7f80b88f4ce2d816e9445599e0841b26e51a6',
  'id': ObjectId('60747e3dd50b3b0b66a6f5ad'),
  'message': 'Going back to the initialization state',
  'parent_commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'snapshot_id': ObjectId('60747e39d50b3b0b66a6f5a3'),
  'timestamp': '2021-04-12T13:07:09.385321',
  'type': 'COMMIT'},
 {'commit_hash': '3db1b4b6eca18264b9eef9b72d69c905b6ebdbfc3114696419ca99888208cece',
  'id': ObjectId('60747e3cd50b3b0b66a6f5aa'),
  'message': 'This is my first commit',
  'parent_commit_hash': 'a8118917361a8ff6f6bd182e31ec

We can also go back to a particular snapshot in history. In this case let's go back to our first commit, which should again empty the database

In [34]:
history = did.session.get_history()
did.session.set_current_ref(snapshot_id=history[len(history)-1]['snapshot_id'])
print_data(my_did.find())

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



Note that this creates a new commit with the message that tells the user the snapshot_id the database has now switched to

In [35]:
did.session.get_history()

[{'commit_hash': 'a97d6634fd9f3f423eb65126367b9330840b23b002f0aa40f28f280530f2e832',
  'id': ObjectId('60747e3ed50b3b0b66a6f5b1'),
  'message': 'Switch to snapshot_id: 60747e39d50b3b0b66a6f5a3',
  'parent_commit_hash': '202f01a4a0fcbabd210907ed0fa76ccfb65218086b7dfc2a634688e9e89202c6',
  'snapshot_id': ObjectId('60747e39d50b3b0b66a6f5a3'),
  'timestamp': '2021-04-12T13:07:10.003915',
  'type': 'COMMIT'},
 {'commit_hash': '202f01a4a0fcbabd210907ed0fa76ccfb65218086b7dfc2a634688e9e89202c6',
  'id': ObjectId('60747e3dd50b3b0b66a6f5b0'),
  'message': 'My third commit!',
  'parent_commit_hash': '27fa5c0dd95905d3d4860bc392f7f80b88f4ce2d816e9445599e0841b26e51a6',
  'snapshot_id': ObjectId('60747e3dd50b3b0b66a6f5ae'),
  'timestamp': '2021-04-12T13:07:09.829421',
  'type': 'COMMIT'},
 {'commit_hash': '27fa5c0dd95905d3d4860bc392f7f80b88f4ce2d816e9445599e0841b26e51a6',
  'id': ObjectId('60747e3dd50b3b0b66a6f5ad'),
  'message': 'Going back to the initialization state',
  'parent_commit_hash': '3db1

### GridFS Demo

Access the GridFS instance using did.session.bin

In [36]:
fs = did.session.bin

Let's store all the mock DIDDocument back to the MongoDB database

In [37]:
for doc in docs:
    did.session.add(doc)
did.session.save("Insert all mock documents back")

Let's try attaching a binary file to one of the document in docs

In [38]:
doc = did.session.find()[0]
print(doc.data)
print()

{'base': {'id': '0', 'session_id': '2387492', 'name': 'A', 'datestamp': '2020-10-28T08:12:20+0000', 'snapshots': ['60747e3ad50b3b0b66a6f5a6'], 'records': ['96fde3fd5202e40fc9f179db871af179cb3f0faf4d787fc9ad0a0883bca6565b']}, 'depends_on': [], 'dependencies': [], 'binary_files': [], 'document_class': {'definition': '$NDIDOCUMENTPATH/ndi_document_app.json', 'validation': '$NDISCHEMAPATH/ndi_document_app_schema.json', 'class_name': 'did_document_app', 'property_list_name': 'app', 'class_version': 1, 'superclasses': [{'definition': '$NDIDOCUMENTPATH/base_document.json'}]}, 'app': {'a': True, 'b': True}}



We can access all the binary files that are already attached to a particular DIDDocument. Noting has been attached to this document yet

In [39]:
fs.list_files(doc)

{}

We can open a write stream and insert byte into the binary file associated with this document. Remeber to close the document when we are done writting.

In [40]:
binary_doc = fs.open_write_stream(doc, 'file1.bin')
binary_doc.fwrite(b'some_data')
binary_doc.fclose()

Now if we run the list_file method, we can see the binary file that is attached to this document

In [41]:
fs.list_files(doc)

{'file1.bin': ObjectId('60747e3fd50b3b0b66a6f5b6')}

We can also open a read stream that allows us to read a portion of the binary file. Run ftell() to get the position of the cursor in the binary file

In [42]:
binary_doc = fs.open_read_stream(doc, 'file1.bin')
binary_doc.ftell()

0

Read 3 bytes further

In [43]:
binary_doc.fread(3)

b'som'

Read the rest of the files

In [44]:
binary_doc.fread()

b'e_data'

Go back to the beginning of the files, and read the entire file at once, then close the binary_doc

In [45]:
binary_doc.fseek(0)
print(binary_doc.fread())
binary_doc.fclose()

b'some_data'


We can also remove the binary file if we no longer need it

In [46]:
fs.remove_file(doc, 'file1.bin')
fs.list_files(doc)

{}

An error will be thrown if we attempt to open a read stream of a file that does not exist

In [47]:
try:
    binary_doc = fs.open_read_stream(doc, 'file1.bin')
except Exception as ex:
    print("Error: {}".format(ex))

Error: File name: file1.bin is not associated with this document, here are the list of associated binary file: {}
